# Daredevil Demo - Info 290: Privacy and Security Lab
---

In this lab, we will explore the potential privacy concerns regarding location data that is supposedly anonymous. We will use a modified version of NYC Taxi data (which is made public and can be found [here](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)) and modified NYC complaints data (found [here](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Map-Year-to-Date-/2fra-mtpn)).

Based on the fictional Marvel superhero Daredevil, we will use these two datasets to find the identity/location of Daredevil (if you do not know the background of the superhero, do not worry). While this is a seemingly trivial example, it turns out that knowing just a little bit of information can be combined with a dataset to discover much more than [intended](https://research.neustar.biz/2014/09/15/riding-with-the-stars-passenger-privacy-in-the-nyc-taxicab-dataset/).

**We will look at past crime data, and knowing that Daredevil is blind and thus cannot drive himself (assume Uber does not yet exist), must use a taxi to reach crimes far from his home**

**This demo assumes a basic knowledge of Python (up to functions and for-loops)**

*Estimated Time: 60 minutes*

---

**Topics Covered:**
- Loading/Processing Data
- Data Visualization
- Combining, Exploring, and Using Data

**Dependencies:**
*if you are running this through DataHub, you do not need to worry about installing these*
- numpy
- datascience
- folium
- datetime


In [1]:
# Just run this cell. It imports all of the packages we will use
import numpy as np
from datascience import *
import folium
import datetime as dt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
!pip install -U folium

Requirement already up-to-date: folium in /srv/venv/lib/python3.6/site-packages
Requirement already up-to-date: branca in /srv/venv/lib/python3.6/site-packages (from folium)
Requirement already up-to-date: jinja2 in /srv/venv/lib/python3.6/site-packages (from folium)
Requirement already up-to-date: requests in /srv/venv/lib/python3.6/site-packages (from folium)
Requirement already up-to-date: six in /srv/venv/lib/python3.6/site-packages (from folium)
Requirement already up-to-date: MarkupSafe>=0.23 in /srv/venv/lib/python3.6/site-packages (from jinja2->folium)
Requirement already up-to-date: urllib3<1.23,>=1.21.1 in /srv/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: idna<2.7,>=2.5 in /srv/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: certifi>=2017.4.17 in /srv/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in /srv/venv/lib/python3.6/site-

*Quick note, if you ever want to know more about a certain function, you can add a **?** after a function name to pull up the docstring for the function*

In [2]:
Table.read_table?

## Loading the data

To start, we will load in the raw csv data (remember we are using 2 datasets) and view each one individually. Observe the column names and try to make note of what each name means. For some datasets, these names can be obscure and you will need to look directly at the source of the data to have more information about each column. However, in our case, most of the columns have column names we can easily interpret. There are a few columns that are not very clear about what they mean, but none of these columns will affect our search for the Daredevil in any significant way so we will ignore them (at least, in our demo)

In [3]:
# The lines below will load the data
taxis_raw = Table.read_table("Taxi_data.csv")
complaints_raw = Table.read_table("NY_complaints.csv")

# Use .show(x) function to show the first x lines of a table
print("Taxi Data:")
taxis_raw.show(5)
print("Complaints Data:")
complaints_raw.show(5)

Taxi Data:


VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type
2,2016-01-16 10:24:29,2016-01-16 10:32:46,N,1,-73.951,40.6753,-73.938,40.6826,1,1.38,7.5,0,0.5,1.66,0,nan,0.3,9.96,1,1
2,2016-02-22 09:23:39,2016-02-22 09:26:08,N,1,-73.9571,40.7458,-73.954,40.7427,1,0.47,3.5,0,0.5,1,0,nan,0.3,5.3,1,1
2,2015-12-01 21:05:00,2015-12-01 21:08:28,N,1,-73.9033,40.7458,-73.9129,40.7485,1,0.63,4.5,0.5,0.5,0,0,nan,0.3,5.8,2,1
2,2016-03-03 10:43:47,2016-03-03 11:14:04,N,1,-73.952,40.6823,-74.0057,40.7405,1,6.12,23,0,0.5,4.76,0,nan,0.3,28.56,1,1
2,2016-03-21 17:44:57,2016-03-21 18:04:18,N,1,-73.977,40.6768,-73.9423,40.6842,1,2.61,13.5,1,0.5,3,0,nan,0.3,18.3,1,1


Complaints Data:


CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
614672714,03/24/2016,16:45:00,03/24/2016,17:00:00,03/26/2016,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,N.Y. POLICE DEPT,BROOKLYN,67,FRONT OF,RESIDENCE-HOUSE,nan,nan,1.00256e+06,172696,40.6407,-73.934,"(40.640676528, -73.93404401)"
864796982,02/22/2016,13:50:00,02/22/2016,14:10:00,02/22/2016,110,GRAND LARCENY OF MOTOR VEHICLE,441,"LARCENY,GRAND OF AUTO",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,46,FRONT OF,STREET,nan,nan,1.00933e+06,250025,40.8529,-73.9093,"(40.852908431, -73.909340056)"
105856897,03/21/2016,14:50:00,03/21/2016,14:55:00,03/21/2016,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,STATEN ISLAND,121,INSIDE,RESIDENCE-HOUSE,nan,nan,944075,166443,40.6234,-74.1447,"(40.623441179, -74.144724551)"
948233328,03/28/2016,18:15:00,03/28/2016,18:20:00,03/28/2016,341,PETIT LARCENY,333,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,BRONX,45,INSIDE,JEWELRY,nan,nan,1.0336e+06,255731,40.8685,-73.8216,"(40.868467378, -73.821582846)"
765467317,02/02/2016,02:00:00,02/02/2016,02:01:00,02/02/2016,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,QUEENS,114,INSIDE,RESIDENCE - APT. HOUSE,nan,nan,1.00678e+06,220990,40.7732,-73.9187,"(40.773222365, -73.91867316)"


We see that the data we have gives a lot of information! In particular, there seems to be a wealth of information in the form of times and locations. This sets up our general approach to finding our target. We will assume that for some number of the crimes committed (there would be no way for Daredevil to get to all crimes), Daredevil must have taken a taxi and was dropped off near the location of the crime. Thus, we can try to determine the taxis Daredevil took and then look at the original pickup location. However, this is complicated by the fact that we have much more data than we want and that we cannot expect Daredevil to have gotten a ride exactly to the same location and at the same exact time.

## "Cleaning" our Data

Before we move on to actually analyzing the data, we must process the data to be in a more usable form. Raw data is often very messy and can be a pain to work with. There can be missing values or [nans](https://en.wikipedia.org/wiki/NaN) (Not A Number) scattered throughout the dataset, and values can often be in a hard to use form. Processing the data now will make our lives much easier later.

To start, we'll make the tables smaller by including only columns of interest. While this helps to focus our analysis, note that this also discards potentially useful information. If you finish the demo early and want to try some of your own analyses, feel free to use more columns than we do here (in creating the mock data, we use many more columns).

For our taxi dataset, we will only select the columns for pickup/dropoff times, pickup/dropoff locations and the passenger count. For our complaints data, we will only select the level of offense (`LAW_CAT_CD`).

In [4]:
# Selecting the columns of Taxi Data according to column index
taxis = taxis_raw.select([1,2,5,6,7,8,9])
taxis.relabel(['lpep_pickup_datetime', 'Lpep_dropoff_datetime'], ['Pickup_dt', 'Dropoff_dt']) # renames column
print("Taxi Data:")
taxis.show(5)

# Selecting the columns of Complaints Data according to column index
complaints = complaints_raw.select([1,2,7,9,11,21,22])
print("Complaints Data:")
complaints.show(5)

Taxi Data:


Pickup_dt,Dropoff_dt,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count
2016-01-16 10:24:29,2016-01-16 10:32:46,-73.951,40.6753,-73.938,40.6826,1
2016-02-22 09:23:39,2016-02-22 09:26:08,-73.9571,40.7458,-73.954,40.7427,1
2015-12-01 21:05:00,2015-12-01 21:08:28,-73.9033,40.7458,-73.9129,40.7485,1
2016-03-03 10:43:47,2016-03-03 11:14:04,-73.952,40.6823,-74.0057,40.7405,1
2016-03-21 17:44:57,2016-03-21 18:04:18,-73.977,40.6768,-73.9423,40.6842,1


Complaints Data:


CMPLNT_FR_DT,CMPLNT_FR_TM,OFNS_DESC,PD_DESC,LAW_CAT_CD,Latitude,Longitude
03/24/2016,16:45:00,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",VIOLATION,40.6407,-73.934
02/22/2016,13:50:00,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",FELONY,40.8529,-73.9093
03/21/2016,14:50:00,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,40.6234,-74.1447
03/28/2016,18:15:00,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",MISDEMEANOR,40.8685,-73.8216
02/02/2016,02:00:00,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,40.7732,-73.9187


Now we have selected our columns, let's remove all of the rows that have a missing or null value. We will also remove zero values because some of the taxi locations have 0,0 as their coordinates (which is [clearly](https://www.google.com/maps/place/0%C2%B000'00.0%22N+0%C2%B000'00.0%22E/) not correct).

In [5]:
def remove_nan(t):
    """
    Removes all rows with nan values checking each column
    Note you should use this AFTER stripping the table of columns you do not need
    so you do not remove rows when given a column without much information

    Will remove most nan values but may not work with some other default missing values
    (specifically, will not remove -999, etc. values)

    Parameters:
    t: a table whose rows with nan values you want to remove

    returns a table identical to t but without rows containing nan values
    """
    def checkNotnan(val):
        if (val!=val)|(val=='nan')|(val=='NAN')|(val=='NaN')|(val==0):
            return False
        return True
    for i in range(t.num_columns):
        t = t.where(i, checkNotnan)
    return t

taxis = remove_nan(taxis)
complaints = remove_nan(complaints)
taxis.show(5)
complaints.show(5)

Pickup_dt,Dropoff_dt,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count
2016-01-16 10:24:29,2016-01-16 10:32:46,-73.951,40.6753,-73.938,40.6826,1
2016-02-22 09:23:39,2016-02-22 09:26:08,-73.9571,40.7458,-73.954,40.7427,1
2015-12-01 21:05:00,2015-12-01 21:08:28,-73.9033,40.7458,-73.9129,40.7485,1
2016-03-03 10:43:47,2016-03-03 11:14:04,-73.952,40.6823,-74.0057,40.7405,1
2016-03-21 17:44:57,2016-03-21 18:04:18,-73.977,40.6768,-73.9423,40.6842,1


CMPLNT_FR_DT,CMPLNT_FR_TM,OFNS_DESC,PD_DESC,LAW_CAT_CD,Latitude,Longitude
03/24/2016,16:45:00,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",VIOLATION,40.6407,-73.934
02/22/2016,13:50:00,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",FELONY,40.8529,-73.9093
03/21/2016,14:50:00,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,40.6234,-74.1447
03/28/2016,18:15:00,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",MISDEMEANOR,40.8685,-73.8216
02/02/2016,02:00:00,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,40.7732,-73.9187


We will now convert the format of some of the columns to be more usable. Currently, the taxi dates are in a string format and we would like to change it to a [datetime](https://docs.python.org/3/library/datetime.html) object. Effectively, we are taking the time, which is stored like any old written text, and converting it to a functional form where we can easily make comparisons to other times.

To do this we will use the `apply` method to run our function for each row in the datetime columns of the taxi data.

In [6]:
# This function will conveniently convert a specific format of string to a datetime
def to_datetime(string_date):
    '''will strip a date in a string format and return a datetime format'''
    if type(string_date) == dt.datetime:
        return string_date
    return dt.datetime.strptime(string_date, '%Y-%m-%d %H:%M:%S')

print("Before:", taxis.column(0))
print()

converted_pickup_col = taxis.apply(to_datetime, 'Pickup_dt')
converted_dropoff_col = taxis.apply(to_datetime, 'Dropoff_dt')
taxis = taxis.with_column('Pickup_dt', converted_pickup_col)
taxis = taxis.with_column('Dropoff_dt', converted_dropoff_col)
print("After:", taxis.column(0))

Before: ['2016-01-16 10:24:29' '2016-02-22 09:23:39' '2015-12-01 21:05:00' ...,
 '2016-02-21 00:11:00' '2015-12-06 18:11:47' '2015-12-26 11:59:12']

After: [datetime.datetime(2016, 1, 16, 10, 24, 29)
 datetime.datetime(2016, 2, 22, 9, 23, 39)
 datetime.datetime(2015, 12, 1, 21, 5) ...,
 datetime.datetime(2016, 2, 21, 0, 11)
 datetime.datetime(2015, 12, 6, 18, 11, 47)
 datetime.datetime(2015, 12, 26, 11, 59, 12)]


Now we will combine the dates and times of the complaints data once again by applying a function -- but now to two columns! The format of the complaints are also strings, and we want to combine them to be a single datetime object.

In [7]:
# The function we will apply to the table. Do not worry too much about the details of it
def combine_date_time(date_string, time_string):
    '''function that takes a date in the format of a string and a 
    time in the format of a string and then combines the two into a new datetime format'''
    if type(date_string) == dt.date:
        date = date_string
    elif type(date_string) == dt.datetime:
        date = date_string.date()
    else:
        date = dt.datetime.strptime(date_string, '%m/%d/%Y').date()
        
    if type(time_string) == dt.time:
        time = time_string
    elif type(date_string) == dt.datetime:
        time = time_string.time()
    else:
        time = dt.datetime.strptime(time_string, '%H:%M:%S').time()
    return dt.datetime.combine(date, time)

# applies the function above
combined_times = complaints.apply(combine_date_time, ["CMPLNT_FR_DT", "CMPLNT_FR_TM"])
complaints = complaints.with_column("Complaint_dt", combined_times)
# drops the first two columns and reorders the table
complaints = complaints.drop([0, 1]).select([5,3,4,0,1,2])
complaints.show(5)

Complaint_dt,Latitude,Longitude,OFNS_DESC,PD_DESC,LAW_CAT_CD
2016-03-24 16:45:00,40.6407,-73.934,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",VIOLATION
2016-02-22 13:50:00,40.8529,-73.9093,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",FELONY
2016-03-21 14:50:00,40.6234,-74.1447,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR
2016-03-28 18:15:00,40.8685,-73.8216,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",MISDEMEANOR
2016-02-02 02:00:00,40.7732,-73.9187,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR


We will also combine the latitude/longitude data into one column so we can more easily apply functions using the `datascience` package.

In [8]:
def combine_coordinates(latitude, longitude):
    """
    returns a tuple of the given latitude and longitude
    """
    return (latitude, longitude)

# Apply the function above
taxis_combined_pickup_loc = taxis.apply(combine_coordinates, ['Pickup_latitude', 'Pickup_longitude'])
taxis_combined_dropoff_loc = taxis.apply(combine_coordinates, ['Dropoff_latitude', 'Dropoff_longitude'])
complaints_combined_loc = complaints.apply(combine_coordinates, ['Latitude', 'Longitude'])

# combine with tables and drop previous columns
taxis = taxis.with_column("Pickup_location", taxis_combined_pickup_loc)
taxis = taxis.with_column("Dropoff_location", taxis_combined_dropoff_loc)
taxis = taxis.drop([2,3,4,5])

complaints = complaints.with_column("Location", complaints_combined_loc)
complaints = complaints.drop([1,2])

taxis.show(2)
complaints.show(2)

Pickup_dt,Dropoff_dt,Passenger_count,Pickup_location,Dropoff_location
2016-01-16 10:24:29,2016-01-16 10:32:46,1,[ 40.67527008 -73.95104218],[ 40.68263626 -73.93795776]
2016-02-22 09:23:39,2016-02-22 09:26:08,1,[ 40.74575424 -73.95707703],[ 40.742733 -73.95397186]


Complaint_dt,OFNS_DESC,PD_DESC,LAW_CAT_CD,Location
2016-03-24 16:45:00,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",VIOLATION,[ 40.64067653 -73.93404401]
2016-02-22 13:50:00,GRAND LARCENY OF MOTOR VEHICLE,"LARCENY,GRAND OF AUTO",FELONY,[ 40.85290843 -73.90934006]


Now we get to move on from the boring (yet important) preprocessing of data!

## Visualization

Before we begin trying to find our DareDevil, we will explore some of the visualization tools that we can use to easily see the data. We will be using `folium` for this purpose. You can look through the `folium` [quickstart guide](https://folium.readthedocs.io/en/latest/) or use some of the built-in helper functions we provide.

In [9]:
# This is the syntax to create an empty map centered at coordinates 40.7127,-74.0059
# This is also the coordinates of NYC so you can simply use these coordinates in any other maps for this lab
map_example = folium.Map(width=700,height=500,location=[40.7128,-74.0059], zoom_start=10)

# to display the map simply type the name
map_example

In order to start plotting points for the lab, `folium` uses a class called Markers. You can read more documentation [here](https://folium.readthedocs.io/en/latest/quickstart.html#markers). The basics of `folium` are displayed below.

In [10]:
# Creating a new marker at coordinates (40.8436, -73.5633)
marker_example = folium.Marker([40.72, -73.9633])
# adds the marker to the map
marker_example.add_to(map_example)
# Note that there is no easy way to remove a marker once you add it to the map
# If you want reset a map, simply run map_example = folium.Map(location=[40.7128,-74.0059])
# in order to create a new one instead

# display the map
map_example

We define a function `addMarkers` below. This function will automatically add markers to a map from a given table assuming the table has a column called "Location". 

In [11]:
def addMarkers(fol_map, mark, location_col, color="blue",icon='star',max_num=25, popup_cols=[]):
    """
    adds markers to folium fol_map based on a table mark
    Parameters:
    fol_map: a folium.Map class that you want to add markers to
    mark: a table containing two columns 'Latitude' and 'Longitude'
        if these columns do not exits, defaults to using first column as latitude and 2nd as longitude
    color: color of the marker added (default: blue)
    icon: icon of marker added (default: star)
    max_num: the maximum number of markers added. Use to not overload folium map (default: 25)
    popup: the columns of the table to be included
    returns nothing. Will modify fol_map directly
    """
    if type(location_col)==str:
        location_col = mark.column_index(location_col)
    for i in range(mark.num_rows):
        row = mark.row(i)
        popup = None
        if len(popup_cols)>0:
            popup = ""
            for col in popup_cols:
                popup += mark.column_labels[col] + ": " + str(row[col]) + '  '
        folium.Marker(row[location_col],icon=folium.Icon(color=color, icon=icon), popup=popup).add_to(fol_map)
        if (i>max_num):
             return

Now we can add markers to the map from our taxi data set!

In [12]:
# reset the map_example variable
map_example = folium.Map(width=700,height=500,location=[40.7128,-74.0059], zoom_start=10)

# You can also change the color and icon of the markers
addMarkers(map_example, taxis, 'Dropoff_location', color='red', icon='cloud', popup_cols=[0,1])
# type help(folium.Icon) to get some details of what you can put in color and icon

map_example

## Analyzing Data

We will be looking at the latitude and longitude data from `complaints` and `taxi` as well as the times of each of those events (so if you dropped these columns earlier, go back and change your selection so these columns are included).

The rationale in our analysis is that DareDevil responds to crimes that show up in the NYPD complaints log, which lists the location of a crime. Thus, if we look at a crime where Daredevil was present, we expect to find a corresponding taxi that goes to the general area. Then, if we look at where this taxi originated from, we should be (in theory) able to find where Daredevil originates from and thus get closer to identifying him.

In the real world, you can imagine that we would use a variety of ideas to begin looking for specific people or narrow our search (e.g. photos of taxis celebrities emerged out of, knowledge of where someone lives, etc.)

In [13]:
# Run this cell to display the tables
taxis.show(1)
complaints.show(1)

Pickup_dt,Dropoff_dt,Passenger_count,Pickup_location,Dropoff_location
2016-01-16 10:24:29,2016-01-16 10:32:46,1,[ 40.67527008 -73.95104218],[ 40.68263626 -73.93795776]


Complaint_dt,OFNS_DESC,PD_DESC,LAW_CAT_CD,Location
2016-03-24 16:45:00,HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",VIOLATION,[ 40.64067653 -73.93404401]


Lets look at the times first. Perhaps we know that DareDevil took some taxi sometime near 11:00 pm (23:00) on January 4th. We can try to find the destination by looking at all taxi rides around that time using the `taxi` dataset. We can then write a function that checks if an event occurs within x minutes from a certain time. Then, we can use this function to select only rows from the tables that correspond to these times.

We will use the `Table.where` method. One way to use this function is to give it a column name and a function that returns a boolean value (True or False) and it returns a copy of the original table, but with only the rows where the function returned True when applied to that column.

In [14]:
def near_11pm_jan_4(time):
    '''
    Returns a boolean (true or false) whether a time is 5 minutes away from January 4, 2016 at 11pm.
    '''
    jan_4_11_pm = dt.datetime(2016,1,4,23)
    return abs(time - jan_4_11_pm) <= dt.timedelta(minutes=5)

# Now we use the .where function to select rows from taxi where the pickup time was within 5 minutes of 11pm
# on January 4th!
near_11_taxis = taxis.where('Pickup_dt', near_11pm_jan_4)
near_11_taxis.show()

Pickup_dt,Dropoff_dt,Passenger_count,Pickup_location,Dropoff_location
2016-01-04 23:02:03,2016-01-04 23:32:31,1,[ 40.83343887 -73.8605957 ],[ 40.75482559 -73.9465332 ]


You may have noticed that this function is pretty restrictive. It only allows us to check a table for one specific time! Below we will define a new function that is more general and will allow us to compare two times with each other. This will prove very useful later.

In [15]:
def time_near(time1, time2, error=5):
    '''
    Returns a boolean (true or false) whether 2 times are within error time of each other
    error time is a number representing the minutes in between the two times (default 5 minutes)
    '''
    return abs(time1-time2) <= dt.timedelta(minutes=error)

# Do not worry exactly what this line below does. It essentially creates the same 
# function we had before (and of the same name) using the more general function
near_11pm_jan_4 = lambda x: time_near(x, dt.datetime(2016,1,4,23))
near_11_taxis = taxis.where('Pickup_dt', near_11pm_jan_4)
near_11_taxis.show()

Pickup_dt,Dropoff_dt,Passenger_count,Pickup_location,Dropoff_location
2016-01-04 23:02:03,2016-01-04 23:32:31,1,[ 40.83343887 -73.8605957 ],[ 40.75482559 -73.9465332 ]


Using this method, we can look at the time of a crime we believe Daredevil to have gone to and look at dropoff or pickup times for taxis around the same time. Below, we will create a list of tables that correspond to the first 5 rows of crimes/complaints. These tables will be all the taxis that have dropoffs within 10 minutes of the crime being reported.

In [16]:
table_list = []
for i in range(5):
    time = complaints.column('Complaint_dt')[i] #gets the ith datetime of the complaint
    temp_function = lambda x: time_near(x, time, 10)
    table_list.append(taxis.where('Dropoff_dt', temp_function))
table_list[4].show()

Pickup_dt,Dropoff_dt,Passenger_count,Pickup_location,Dropoff_location
2016-02-02 01:40:39,2016-02-02 01:53:35,1,[ 40.68479156 -73.97782135],[ 40.66083908 -73.9831543 ]


Now lets look at location data. As with time, we can create a function to let us get all of the rows of a table with a location that is close to some certain coordinate. 

Before we do, we write a function that converts (roughly) the distance in km of two coordinates to make our lives easier.

In [17]:
def dist_coord(loc1,loc2):
    """
    returns distance in km between 2 coordinates
    loc1 and loc2 should be a tuple of coordinates corresponding to the latitude and longitudes
    of 2 locations
    Not entirely accurate (assumes perfectly spherical earth) but works for our purposes
    """
    R = 6373.0
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (np.sin(dlat/2))**2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2))**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance

And now we write a general function (like we did for time) to be used in `.where` call of a table. That is, we will write a function that returns True if the coordinates are within x km of another set coordinate. We use the function to then generate a table of taxis that dropped off a passenger within 1 km of the first complaint location (40.638, -73.8985)

In [18]:
def dist_near(loc1, loc2, error=1):
    """
    returns a boolean (True or False) of whether the coordinates (lat1,lon1) and (lat2,lon2) are
    within error km (default 1 km) of each other
    """
    return dist_coord(loc1, loc2) <= error

complaint_loc = complaints.column('Location')[0]
near_first_complaint_func = lambda x: dist_near(x, complaint_loc)
near_first_complaint_table = taxis.where('Dropoff_location', near_first_complaint_func)
near_first_complaint_table.show()

Pickup_dt,Dropoff_dt,Passenger_count,Pickup_location,Dropoff_location
2016-01-04 11:32:46,2016-01-04 11:33:03,1,[ 40.63484955 -73.93488312],[ 40.63485718 -73.93489075]
2015-12-13 15:42:45,2015-12-13 15:49:43,1,[ 40.65153885 -73.95597839],[ 40.64172363 -73.9371109 ]
2016-01-30 16:43:17,2016-01-30 16:59:45,1,[ 40.67980576 -73.92873383],[ 40.64642715 -73.92745209]
2016-03-29 20:30:07,2016-03-29 20:40:20,1,[ 40.63330078 -73.94945526],[ 40.63592148 -73.93845367]
2015-12-04 18:46:34,2015-12-04 19:07:52,1,[ 40.6841011 -73.97599792],[ 40.64549637 -73.93251801]
2016-02-10 02:52:44,2016-02-10 02:55:02,1,[ 40.63214111 -73.94717407],[ 40.63244629 -73.93502045]
2016-02-28 21:19:52,2016-02-28 21:31:19,1,[ 40.63476562 -73.95101166],[ 40.64136505 -73.94203186]
2016-01-06 09:26:04,2016-01-06 09:47:37,2,[ 40.69321823 -73.96798706],[ 40.63546371 -73.94022369]
2015-12-15 03:16:12,2015-12-15 03:23:31,1,[ 40.63249207 -73.94744873],[ 40.63758469 -73.9230957 ]
2016-03-29 15:23:45,2016-03-29 15:25:01,1,[ 40.64242935 -73.94282532],[ 40.64245605 -73.94281006]


Let's visualize this! We plot the location of the complaint in red, and the locations of the taxi dropoffs in green. In addition, we plot the pickup location for each of these taxis in blue so you can see where the taxis picked up passengers who were dropped near the location.

In [19]:
distance_example_map = folium.Map(width=700,height=500,location=complaint_loc.tolist(), zoom_start=12)

addMarkers(distance_example_map, near_first_complaint_table, 'Dropoff_location', color='green', popup_cols=[3,4])
addMarkers(distance_example_map, near_first_complaint_table, 'Pickup_location', color='blue', popup_cols=[3,4])
folium.Marker(complaint_loc, icon=folium.Icon(color='red'), popup='Complaint_location: '+str(complaint_loc)).add_to(distance_example_map)

distance_example_map

Now let's try to find DareDevil! We will try to go through the complaints data and find the complaints we believe (or know) that Daredevil has gone to. Then using this, we will match the crime with a taxi that is near in time and dropoff location. Finally, we will look for common pickup locations and suspect that this is the origin location of Daredevil.

For the sake of speed, we will require that the passenger count in taxis be 1 and use only January's data. We also will set the location to 1 km away and the dropoff time to be within 15 minutes of the complaint time. You are free to change any of these parameters as you see fit. In addition, if you are comfortable, try to experiment by including some of the columns we removed earlier in the pre-processing stage! 

*Note that some of the visualizations can be time-consuming, so we encourage you to try to find a faster way that does not need visualizations (hint: try to use the distance function on the pickup locations you get).*

In [21]:
distance_err = 1
time_err = 15

felonies = complaints.where("LAW_CAT_CD", "FELONY")
# We create a new table with only felonies
felonies = complaints.where("Complaint_dt", 
        are.between_or_equal_to(dt.datetime(2016,1,1), dt.datetime(2016,1,31)))
felonies.show(5)

# taxi_tables will be an array of tables corresponding to the table of taxis that
# are suspected to be related to the associated felony
taxi_tables = []
for row_num in range(near_first_complaint_table.num_rows):
    # This should all look familiar
    complaint_loc = near_first_complaint_table.column('Pickup_location')[row_num]
    complaint_desc = near_first_complaint_table.column('Dropoff_location')[row_num]
    complaint_dt = near_first_complaint_table.column('Pickup_dt')[row_num]
    
    near_complaint_loc_func = lambda x: dist_near(x, complaint_loc, distance_err)
    near_complaint_time_func = lambda x: time_near(x, complaint_dt, time_err)
    
    temp_table = taxis.where('Dropoff_location', near_complaint_loc_func)
    temp_table = temp_table.where('Dropoff_dt', near_complaint_time_func)
    
    taxi_tables.append(temp_table)
    

# For visualization. Note that this can take awhile to run
NY_map = folium.Map(width=700, height=500, location=[40.7128,-74.0059], zoom_start=10)

for i in range(len(taxi_tables)):
    table = taxi_tables[i]
    if (table.num_rows>0):
        complaint_loc = felonies.column('Location')[i]
        complaint_desc = felonies.column('OFNS_DESC')[i]
        complaint_dt = felonies.column('Complaint_dt')[i]
        
        felony_marker = folium.Marker(complaint_loc, icon=folium.Icon(color='red'), popup=complaint_desc)
        felony_marker.add_to(NY_map)
        addMarkers(NY_map, table, 'Dropoff_location', color='green', popup_cols=[3,4])
        addMarkers(NY_map, table, 'Pickup_location', color='blue', popup_cols=[3,4])
        
NY_map

Complaint_dt,OFNS_DESC,PD_DESC,LAW_CAT_CD,Location
2016-01-24 03:55:00,PETIT LARCENY,"LARCENY,PETIT FROM BUILDING,UN",MISDEMEANOR,[ 40.75288388 -73.98343469]
2016-01-11 06:05:00,HARRASSMENT 2,"HARASSMENT,SUBD 1,CIVILIAN",VIOLATION,[ 40.6146244 -74.08471673]
2016-01-27 19:30:00,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",FELONY,[ 40.8291633 -73.93727189]
2016-01-17 23:09:00,CRIMINAL MISCHIEF & RELATED OF,"TAMPERING 1,CRIMINAL",FELONY,[ 40.74978011 -73.98778087]
2016-01-12 18:19:00,PETIT LARCENY,"LARCENY,PETIT FROM OPEN AREAS,",MISDEMEANOR,[ 40.69086637 -73.98584806]


Remember that in the visualization, red indicates the crime, green indicates the dropoff location, and blue indicates the pickup location. Do you see anywhere that seems to be the origin of many of the blue clusters? In particular you should find that there is a small cluster of blue near the coordinates (40.76, -74.00).

However, how can we be sure that this did not occur by random coincidence? There could be a variety of factors that impact the data (i.e. population, time, tourist-destination, etc). We recommend trying to explore the data yourself to find out more! You might even find more than just the location of Daredevil.

---

## Credits
This module was created as part of the DSEP Modules team for the Spring 2018 offering of INFO 290.

---